In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from googleapiclient.discovery import build


import httpx

class CustomHTTPClient(httpx.Client):
    def __init__(self, *args, **kwargs):
        kwargs.pop("proxies", None) # Remove the 'proxies' argument if present
        super().__init__(*args, **kwargs)

load_dotenv()
client = OpenAI(api_key=os.environ['API_KEY'], http_client=CustomHTTPClient())
model_name = "gpt-3.5-turbo-0125"

google_cse_id = os.environ['GOOGLE_CSE_ID']
google_api_key = os.environ['GOOGLE_API_KEY']

In [2]:
import json

# 뉴스를 리턴하는 함수 작성
def get_search_result(keyword, when, unit="d"):
    service = build("customsearch", "v1", developerKey=google_api_key)
    result = service.cse().list(q=keyword, dateRestrict=f"{when}{unit}", cx=google_cse_id).execute()

    # 검색결과의 상위 3위을 배열에 취득함
    result_list = []
    for i, item in enumerate(result["items"]):
        result_list.append({"title": f'{item["title"]} {item["snippet"]}'})
        if i>=2:
            break

    return json.dumps({"result": result_list})

In [3]:
# 테스트 코드
ret = get_search_result("서울역 이벤트", 1, "m")
ret2 = json.loads(ret)
for n in ret2["result"]:
    print("-"*20)
    print(n["title"])

--------------------
문화역서울284 문화역서울284는 한국 근·현대사의 주요 무대이자 교통과 교류의 관문이었던 구 서울역의 원형을 복원한 복합문화공간입니다. 문화·예술의 창작과 교류가 이루어지는 ...
--------------------
토이저러스 제타플렉스 서울역 오픈 기념 [건프라 이벤트 안내 ... Sep 13, 2023 ... 토이저러스 제타플렉스 서울역 오픈 기념 [건프라 이벤트 안내] ... 9월 14일(목) 오픈 예정인 토이저러스 제타플렉스 서울역 오픈 기념 건프라 판매 행사가 ...
--------------------
Hoxy.. 파이브가이즈 서울역 4호점 오픈한 소식 모르는 분 없으시죠 ... Apr 10, 2024 ... ... 파이브가이즈고속터미널 #파이브가이즈서울역 #이벤트 #댓글이벤트 #fiveguys #fiveguyskorea #fiveguysfans #event".


In [4]:
from openai.types.chat import ChatCompletionToolParam

# 툴 정의
tools=[
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "지정한 키워드의 검색결과를 취득",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "キーワード",
                    },
                    "when": {
                        "type": "number",
                        "description": "날짜나시간의 범주"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["m", "d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    })
]

In [5]:
question = "서울역의 이벤트에 대해서, 최근 1개월이내의 검색결과를 가르쳐줘"

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)

In [6]:
response

ChatCompletion(id='chatcmpl-BKH0AulnM3is4lOyqV7vQ714ZiVuq', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_w59vb2IomY1ur9mE1TrWNgD4', function=Function(arguments='{"keyword":"서울역 이벤트","when":1,"unit":"m"}', name='get_search_result'), type='function')]))], created=1744171614, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=32, prompt_tokens=138, total_tokens=170, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [7]:
response.choices[0].message.tool_calls[0].function.arguments

'{"keyword":"서울역 이벤트","when":1,"unit":"m"}'

In [8]:
# 모델이 툴호출인지 판단함
if response.choices[0].message.tool_calls is not None:
    # 언어 모델의 응답으로부터 툴 취득
    tool = response.choices[0].message.tool_calls[0]

    # 함수명 취득
    function_name = tool.function.name
    print(f"함수명：{function_name}")
    # 인수 취득
    arguments = json.loads(tool.function.arguments)
    print(f"인수：{arguments}")

    # 함수명으로 실행할 함수 판단
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )
    print(f"함수의 실행 결과：{function_response}")

    # 함수의 실행 결과를 messages에 추가하여 언어 모델 호출
    response2 = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )

    # 언어 모델로부터 응답을 출력
    print(response2.choices[0].message.content.strip())

else:
    # 함수호출을 하지 않는다면, 단순히 언어 모델로부터 응답 출력
    print(response.choices[0].message.content.strip())

함수명：get_search_result
인수：{'keyword': '서울역 이벤트', 'when': 1, 'unit': 'm'}
함수의 실행 결과：{"result": [{"title": "\ubb38\ud654\uc5ed\uc11c\uc6b8284 \ubb38\ud654\uc5ed\uc11c\uc6b8284\ub294 \ud55c\uad6d \uadfc\u00b7\ud604\ub300\uc0ac\uc758 \uc8fc\uc694 \ubb34\ub300\uc774\uc790 \uad50\ud1b5\uacfc \uad50\ub958\uc758 \uad00\ubb38\uc774\uc5c8\ub358 \uad6c \uc11c\uc6b8\uc5ed\uc758 \uc6d0\ud615\uc744 \ubcf5\uc6d0\ud55c \ubcf5\ud569\ubb38\ud654\uacf5\uac04\uc785\ub2c8\ub2e4. \ubb38\ud654\u00b7\uc608\uc220\uc758 \ucc3d\uc791\uacfc \uad50\ub958\uac00 \uc774\ub8e8\uc5b4\uc9c0\ub294\u00a0..."}, {"title": "\ud1a0\uc774\uc800\ub7ec\uc2a4 \uc81c\ud0c0\ud50c\ub809\uc2a4 \uc11c\uc6b8\uc5ed \uc624\ud508 \uae30\ub150 [\uac74\ud504\ub77c \uc774\ubca4\ud2b8 \uc548\ub0b4 ... Sep 13, 2023 ... \ud1a0\uc774\uc800\ub7ec\uc2a4 \uc81c\ud0c0\ud50c\ub809\uc2a4 \uc11c\uc6b8\uc5ed \uc624\ud508 \uae30\ub150 [\uac74\ud504\ub77c \uc774\ubca4\ud2b8 \uc548\ub0b4] ... 9\uc6d4 14\uc77c(\ubaa9) \uc624\ud508 \uc608\uc815\uc778 \ud1a0\u

In [9]:
import json
import requests
from bs4 import BeautifulSoup

# 뉴스를 검색하는 함수
def get_news(keyword, when, unit="d"):
    # Web 사이트의 URL 취득
    url=f'https://news.google.com/search?q={keyword} when:{when}{unit}&hl=ko&gl=KR&ceid=KR:ko'

    # Requests를 이용하여 Web페이지 취득
    response = requests.get(url)
    html = response.text

    # BeautifulSoup 를 이용하여 Web 페이지 해석
    soup = BeautifulSoup(html, 'html.parser')

    # soup.select를 이용하여, 뉴스의 제목 취득
    elements = soup.select('article div div div a')

    # 뉴스의 상위 3개 항목을 배열에 취득
    news = []
    for i, element in enumerate(elements):
        news.append({"title": element.getText()})
        if i >= 2:
            break
    return json.dumps({"news":news})

In [10]:
# 테스트용 코드
ret = get_news("봄꽃축제", 5)
ret2 = json.loads(ret)
for n in ret2["news"]:
    print("-" * 20)
    print(n["title"])

--------------------
영등포구의회, 여의도봄꽃축제 상황실 방문
--------------------
영등포구, 여의도 봄꽃축제 연계…‘봄꽃 할인 행사’ 진행
--------------------
화순군, 봄꽃 축제장서 '군민행복 아카데미' 진행


In [11]:
from openai.types.chat import ChatCompletionToolParam

# 툴의 정의
tools=[
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "지정한 검색결과의 키워드를 취득함",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "キーワード",
                    },
                    "when": {
                        "type": "number",
                        "description": "날짜나 시간의 범주"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["m", "d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    }),
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_news",
            "description": "지정한 키워드의 뉴스를 취득",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "키워드",
                    },
                    "when": {
                        "type": "number",
                        "description": "날짜나 시간의 범주"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["m", "d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    })
]

In [12]:
# question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
question = "봄꽃축제에 관한 5일내의 뉴스에 대해 알려줘"

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)
response

ChatCompletion(id='chatcmpl-BKH0tyRtE7zd8m8vOR4sahqcyXpFU', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_FTnajhhFX0n1BLwJdUT2SaSB', function=Function(arguments='{"keyword":"봄꽃축제","when":5,"unit":"d"}', name='get_news'), type='function')]))], created=1744171659, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=31, prompt_tokens=207, total_tokens=238, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [13]:
response.choices[0].message.tool_calls[0].function.arguments

'{"keyword":"봄꽃축제","when":5,"unit":"d"}'

In [14]:
# 모델의 도구호출인지 판단
if response.choices[0].message.tool_calls is not None:
    # 모델의 회답으로부터 도구 취득
    tool = response.choices[0].message.tool_calls[0]

    # 함수명 취득
    function_name = tool.function.name
    # 인수 취득
    arguments = json.loads(tool.function.arguments)

    # 함수명으로 실행할 함수 취득
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )
    elif function_name == "get_news":
        function_response = get_news(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )

    # 함수의 실행 결과를 messages에 추가하여 모델을 취득
    response2 = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )

    # 언어 모델로부터 회답을 출력
    print(response2.choices[0].message.content.strip())

else:
    # 함수호출이 아니라면, 단순히 언어 모델로부터 출력
    print(response.choices[0].message.content.strip())

최근 5일 내에 관련된 봄꽃축제 뉴스는 다음과 같습니다:
1. 영풍문고의회, 내일동사리포상동질 출발
2. 영풍문고, 내일동초 포상동질 평평…'포상 동초 각인 경영' 존경
3. 화천군, 포상동 축제장소 '국민활기 올리는 앵콜' 존경
